In [229]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import copy

PATH = {}

In [230]:
df= pd.read_csv('in_table.csv')
sample =  pd.read_csv('sample.csv')

In [231]:
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8
0,0,"тмметки 0 взятии крови (плазмы,",клеток крови),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Дата,Вид,Кол-во,Дата,Вид,Кол-во,Дата,Вид,Кол-во
2,2,дон-ва,дон-ва,дон-ва,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,2,3,4,5,6,7,8,NaN
4,4,01.04.2015,кр/д (бв),450,11.10.2017,крод (бв),450,16.10.2018,кр/д (бв),450
5,5,27.07.2017,#р/д (ба),450,30.07.2018,кр/д (Ов),450,15.10.2019,кр/д (бв),450


In [232]:
sample.head()

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка
0,87330,141899,Цельная кровь,21.06.2019,18.11.2020,Безвозмездно,"Россия, Татарстан, Альметьевск",703,Принята,Да
1,213527,141899,Плазма,29.08.2019,16.05.2023,Безвозмездно,"Россия, Татарстан, Альметьевск",703,Принята,Да
2,213528,141899,Плазма,30.09.2019,16.05.2023,Безвозмездно,"Россия, Татарстан, Альметьевск",703,Принята,Да
3,213529,141899,Плазма,01.11.2019,16.05.2023,Безвозмездно,"Россия, Татарстан, Альметьевск",703,Принята,Да
4,213530,141899,Плазма,02.12.2019,16.05.2023,Безвозмездно,"Россия, Татарстан, Альметьевск",703,Принята,Да


In [233]:
df_out = pd.DataFrame(columns=sample.columns)
df_out

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка


In [234]:
#functions
#data in column
def convert_to_date(value):
    try:
        return str(datetime.strptime(value, '%d.%m.%Y').date()).replace('-','.')
    except ValueError:
        return np.nan
#type of blood
def correct_type_blood(string):
    plazma = {'пл', 'л', 'п' }
    blood = {'кр', 'р', 'к'}
    substrings = re.findall(r'(?=(\w+))', s)
    if len(set(substrings) & plazma) > 0:
        string = 'Плазма'
    elif len(set(substrings) & blood) > 0:
        string = 'Цельная кровь'
    else:
      string = 'Тромбоциты'
    return string

In [235]:
warnings.filterwarnings("ignore")
# тут тонкое место, если так не разложаться колоноки с нейронки, то попа
df_1 = df[['0','1','2']]
df_2 = df[['3','4','5']]
df_3 = df[['6','7','8']]

df_list = [df_1, df_2, df_3]
df_columns = ['Дата донации','Класс крови','v']

#приводим к единооборазию и безобразию и обрабатываем все
for i in df_list:
    i.columns=df_columns
    #доп колонка с указанием платно или безвозмезндо
    i['Тип донации'] = 'Безвозмездно'
    # в стринг все значения столбца с датой и типом крови
    i[df_columns[0]] = i[df_columns[0]].astype('str')
    i[df_columns[1]] = i[df_columns[1]].astype('str')
    # переводим в дату и обратнов стринг в нужном формате, если не получиться = nan
    i[df_columns[0]] = i[df_columns[0]].map(convert_to_date)
    #выкидываем строки с nan  для даты, тем самым все очищается от шлака вначале таблицы
    i.dropna(subset=[df_columns[0]], inplace=True)
    #тип сдачи материала(крвоь плазма тромбо)
    i[df_columns[1]] = i[df_columns[1]].map(correct_type_blood)
    df_out = pd.concat([df_out, i.drop('v',axis=1)], axis=0)
df_out['Есть справка'] = 'Да'
df_out['Статус донации'] = 'Принято'
df_out

,ID,ID пользователя,Класс крови,Дата донации,Дата добавления донации,Тип донации,Регион,Место стадчи,Статус донации,Есть справка
4,NaN,NaN,Плазма,2015.04.01,NaN,Безвозмездно,NaN,NaN,Принято,Да
5,NaN,NaN,Плазма,2017.07.27,NaN,Безвозмездно,NaN,NaN,Принято,Да
4,NaN,NaN,Плазма,2017.10.11,NaN,Безвозмездно,NaN,NaN,Принято,Да
5,NaN,NaN,Плазма,2018.07.30,NaN,Безвозмездно,NaN,NaN,Принято,Да
4,NaN,NaN,Плазма,2018.10.16,NaN,Безвозмездно,NaN,NaN,Принято,Да
5,NaN,NaN,Плазма,2019.10.15,NaN,Безвозмездно,NaN,NaN,Принято,Да


In [242]:
df_out.to_csv('out_table.csv', index=True)

'/content'